In [11]:
import sqlite3
import pandas as pd


# Cria banco
conn = sqlite3.connect("campeonato.db")
cur = conn.cursor()

In [12]:
# Apaga tabelas se existirem
cur.execute("DROP TABLE IF EXISTS times;")
cur.execute("DROP TABLE IF EXISTS jogos;")

In [13]:
# Cria as tabelas
cur.execute("""
CREATE TABLE times (
    time_id INTEGER NOT NULL,
    time_nome TEXT NOT NULL,
    UNIQUE(time_id)
)
""")

cur.execute("""
CREATE TABLE jogos (
    jogo_id INTEGER NOT NULL,
    time_mandante INTEGER NOT NULL,
    time_visitante INTEGER NOT NULL,
    gols_mandante INTEGER NOT NULL,
    gols_visitante INTEGER NOT NULL,
    UNIQUE(jogo_id)
)
""")

# Insere os dados
cur.executemany("INSERT INTO Times VALUES (?,?)", [
    (10, 'Financeiro'),
    (20, 'Marketing'),
    (30, 'Logística'),
    (40, 'TI'),
    (50, 'Dados')
])

cur.executemany("INSERT INTO Jogos VALUES (?,?,?,?,?)", [
    (1, 30, 20, 1, 0),
    (2, 10, 20, 1, 2),
    (3, 20, 50, 2, 2),
    (4, 10, 30, 1, 0),
    (5, 30, 50, 0, 1)
])

conn.commit()

In [14]:
query = """
  SELECT 
    time_mandante AS time_id,
    CASE
      WHEN gols_mandante > gols_visitante THEN 3
      WHEN gols_mandante = gols_visitante THEN 1
      ELSE 0
    END AS pontos
  FROM jogos
"""

# Executa e mostra resultado
df = pd.read_sql_query(query, conn)
print(df)

   time_id  pontos
0       30       3
1       10       0
2       20       1
3       10       3
4       30       0


In [15]:
query = """
    SELECT 
        time_visitante AS time_id,
        CASE
        WHEN gols_visitante > gols_mandante THEN 3
        WHEN gols_visitante = gols_mandante THEN 1
        ELSE 0
        END AS pontos
    FROM jogos
"""

# Executa e mostra resultado
df = pd.read_sql_query(query, conn)
print(df)

   time_id  pontos
0       20       0
1       20       3
2       50       1
3       30       0
4       50       3


In [17]:
query = """
  SELECT 
    time_mandante AS time_id,
    CASE
      WHEN gols_mandante > gols_visitante THEN 3
      WHEN gols_mandante = gols_visitante THEN 1
      ELSE 0
    END AS pontos
  FROM Jogos

  UNION ALL

  SELECT 
    time_visitante AS time_id,
    CASE
      WHEN gols_visitante > gols_mandante THEN 3
      WHEN gols_visitante = gols_mandante THEN 1
      ELSE 0
    END AS pontos
  FROM Jogos
"""

# Executa e mostra resultado
df = pd.read_sql_query(query, conn)
print(df)

   time_id  pontos
0       30       3
1       10       0
2       20       1
3       10       3
4       30       0
5       20       0
6       20       3
7       50       1
8       30       0
9       50       3


In [19]:
query = """
WITH Resultados AS (
  SELECT 
    time_mandante AS time_id,
    CASE
      WHEN gols_mandante > gols_visitante THEN 3
      WHEN gols_mandante = gols_visitante THEN 1
      ELSE 0
    END AS pontos
  FROM jogos
  
  UNION ALL

  SELECT 
    time_visitante AS time_id,
    CASE
      WHEN gols_visitante > gols_mandante THEN 3
      WHEN gols_visitante = gols_mandante THEN 1
      ELSE 0
    END AS pontos
  FROM jogos
)

SELECT 
  T.time_id,
  T.time_nome,
  COALESCE(SUM(R.pontos), 0) AS num_pontos
FROM times T
LEFT JOIN Resultados R ON T.time_id = R.time_id
GROUP BY T.time_id, T.time_nome
ORDER BY Num_pontos DESC, T.time_id
"""

# Executa e mostra resultado
df = pd.read_sql_query(query, conn)
print(df)


   time_id   time_nome  num_pontos
0       20   Marketing           4
1       50       Dados           4
2       10  Financeiro           3
3       30   Logística           3
4       40          TI           0
